In [8]:
import json 
from pat_sql import PatJob
from pat_common import AppSettings

# Submit Job

In [3]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

{'job_guid': '80a1ca07-41f9-4f34-b7bb-d01b477bc78d',
 'job_name': 'PAT_Test_AIG',
 'error_action': 'continue',
 'type_of_rating': 'PSOLD',
 'coverage': 'Building + Contents + Time Element',
 'peril_subline': 'All_Perils',
 'subject_premium': 100000000.0,
 'loss_alae_ratio': 1,
 'average_accident_date': '1/1/2022',
 'trend_factor': 1.035,
 'additional_coverage': 2,
 'deductible_treatment': 'Retains Limit',
 'server': 'DFWCAT-RMS5SQL1',
 'edm_database': 'RMS_EPL_AIG_PAT_Testing_202107_EDM181',
 'rdm_database': 'RMS_EPL_AIG_PAT_Testing_202107_RDM181',
 'portinfoid': 6,
 'perilid': 5,
 'analysisid': 5,
 'user_name': 'cxiao',
 'user_email': 'chengyou.xiao@guycarp.com'}

# Run Sequential Code 

In [ ]:
%%time

job = PatJob(job_para) 
if job.job_id> 0:
    job.perform_analysis()    

ret = PatJob.get_results(job.job_id)
res = None
if ret is not None:
    res = pd.DataFrame(json.loads(ret))


# Run with Service

In [14]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}

job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())

ret = requests.post("http://frkcat-ws10:8002/api/Jobs", json=job_para )
ret.status_code, ret.content

(200, b'Analysis submitted: 100000136')

# Results Summary

In [20]:
import requests
import pandas as pd

job_id = 100000132
actions = {
    'list': 'api/Jobs',
    'para': f'api/Jobs/{job_id}/Para',
    'summary': f'api/Jobs/{job_id}',
    'result': f'api/Jobs/{job_id}/Result',
}

ret = requests.get(f"http://frkcat-ws10:8002/{actions['summary']}")
try:
    df = pd.DataFrame(ret.json())
except:
    df = None
    print(ret.content)

df
# print(json.dumps(ret.json(), indent=4))

,cnt_policy,cnt_location,cnt_fac
0,20996,20996,191647


{
    "job_guid": "bd466a3e-ad62-4f21-8f61-566638d28874",
    "job_name": "PAT_Test_AIG",
    "error_action": "continue",
    "type_of_rating": "PSOLD",
    "coverage": "Building + Contents + Time Element",
    "peril_subline": "All_Perils",
    "subject_premium": 100000000.0,
    "loss_alae_ratio": 1,
    "average_accident_date": "1/1/2022",
    "trend_factor": 1.035,
    "additional_coverage": 2,
    "deductible_treatment": "Retains Limit",
    "server": "DFWCAT-RMS5SQL1",
    "edm_database": "RMS_EPL_AIG_PAT_Testing_202107_EDM181",
    "portinfoid": 6,
    "perilid": 5,
    "user_name": "cxiao",
    "user_email": "chengyou.xiao@guycarp.com"
}


In [25]:
df.groupby(by='LocationIDStack').agg({'Premium':'sum'}) 

,Premium
LocationIDStack,
1508370,21127.570639
1508371,23439.639886
1508372,22093.713362
1508373,23439.657574
1508374,10573.501192
...,...
723115,0.000000
723116,0.000000
723117,0.000000
